In [1]:
import copy
import json
import math
import fiona
import folium
import branca.colormap as cm
import requests
import numpy as np
import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import urllib.request
from urllib import request
from shapely.geometry import shape, Point
import os
import threading
from rtree import index
import time
import gzip
from pandas.io.json import json_normalize
from pandas import to_datetime
import pytz
from datetime import datetime, timezone

In [2]:
#========================================================================================================
#爬蟲 (環保署測站、時間、風力資訊、台中各區天氣) + idw

    
ses = requests.Session()
data1 = ses.get('http://taqm.epb.taichung.gov.tw/TQAMNEWAQITABLE.ASPX') #環保署16筆測站
data1.encoding = 'utf-8'
t = pd.read_html(data1.text)[0]
t.drop(t.iloc[:, 1:21], inplace=True, axis=0)
t.drop(t.iloc[:, 1:279], inplace=True, axis=1)
times = str(t[0])



In [3]:
'''#爬蟲 開json.gz

url = "https://pm25.lass-net.org/data/last-all-epa.json.gz"
url1 ="https://pm25.lass-net.org/data/last-all-probecube.json.gz"
gzflieSaveName="last-all-probecube.json.gz"
r = requests.get(url1)
open(gzflieSaveName, 'wb').write(r.content)

f = gzip.open(gzflieSaveName,'r')
jdata = f.read()
f.close()
json_data = json.loads(jdata)  
#json_data
df = pd.json_normalize(json_data,'feeds')
df



#df = json_normalize(json_data)
#df'''

'#爬蟲 開json.gz\n\nurl = "https://pm25.lass-net.org/data/last-all-epa.json.gz"\nurl1 ="https://pm25.lass-net.org/data/last-all-probecube.json.gz"\ngzflieSaveName="last-all-probecube.json.gz"\nr = requests.get(url1)\nopen(gzflieSaveName, \'wb\').write(r.content)\n\nf = gzip.open(gzflieSaveName,\'r\')\njdata = f.read()\nf.close()\njson_data = json.loads(jdata)  \n#json_data\ndf = pd.json_normalize(json_data,\'feeds\')\ndf\n\n\n\n#df = json_normalize(json_data)\n#df'

In [4]:
#爬蟲 微型感測器

url = 'https://pm25.lass-net.org/AirBox/other-epaiot.json'
json_data = request.urlopen(url).read().decode("utf-8")
json_data = json.loads(json_data);
frame= pd.json_normalize(json_data,'feeds')


frame['areaname']=''
frame.index= range(1,len(frame) + 1)
frame['Id']=frame.index
frame.index= range(0,len(frame))
times_micro = frame['latest']
frame.rename(columns={"gps_lon":"longitude"}, inplace=True)
frame.rename(columns={"gps_lat":"latitude"}, inplace=True)

frame[frame['pm25']<0]=np.nan
frame['pm25'].replace({"NA":np.nan}, inplace=True)
frame['pm25'].astype('float64')
frame['pm25'].replace({"-99":np.nan,"-99.0":np.nan}, inplace=True)
frame.dropna(inplace=True)
frame.index= range(0,len(frame))
tz = pytz.timezone('Asia/Taipei')
frame['latest']=to_datetime(frame.latest,format="%Y/%m/%d %H:%M:%S")
#frame['latest']=pd.to_datetime(frame['latest'])

dateTime=frame['latest']



In [82]:
#找出位於台中的微型感測器
def get_areanamebythread(totaldata):
    totalnum=totaldata  #總執行次數
    Q=int(totalnum/10) #取商數
    R=totalnum%10    #取餘數
    
    for i in range(Q):
        threads = []
        for j in range(10):
            threads.append(threading.Thread(
                target=getareaname,#要執行函數
                args=(frame['longitude'][i*10+j],frame['latitude'][i*10+j],i*10+j)))#要執行函數的參數
            threads[j].start()
        for j in threads:
            j.join()
        print(round(float((i+1)*100/totalnum*10),2),'%')#顯示進度
    
    threads = []
    for i in range(R):
        threads.append(threading.Thread(
            target=getareaname,
            args=(frame['longitude'][Q*10+i],frame['latitude'][Q*10+i],Q*10+i)))
        threads[i].start()
    for j in threads:
        j.join()
    print("100%")#顯示進度

def search(x, y):  #尋找鄉鎮
    global shapes, townnames
    #idx = index.Index()
    #for town_id, shape in shapes.items():
    #idx.insert(town_id,shape.bounds)
    return next((townnames[town_id]  #如果鄉鎮區域包含傳入的經緯度就傳回townnames[town_id]
                 for town_id in shapes #idx.intersection((x,y)) #逐一尋找各鄉鎮
                 if shapes[town_id].contains(Point(x, y))), None)

def getareaname(lng,lat,num):
   
    lng = float(lng)
    lat = float(lat)
    
    area_append['areaname'][num]=search(float(lng), float(lat))
    
    return 0

area_append = pd.DataFrame(columns=['areaname'])
area_append['areaname'] = frame['areaname']

module_dir = os.path.dirname('E:\\GitHub\\Independent-study\\web\\123321\\areasearch\\')  #取得目前目錄
collection = fiona.open(os.path.join(module_dir,'TOWN_MOI_1070516.shp'))
shapes = {}
townnames = {}

for f in collection:
    town_id =f['properties']['TOWNCODE'] #鄉鎮代碼
    shapes[town_id] = shape(f['geometry'])  #鄉鎮界限經緯度
    townnames[town_id] = f['properties']['COUNTYNAME'] + f['properties']['TOWNNAME']#search函式傳回值
    


t1=time.time()
get_areanamebythread(frame.shape[0])
t2=time.time()
print('gevent-time:%s' % str(t2-t1))
frame['areaname']=area_append['areaname']

0.13 %
0.26 %
0.39 %
0.52 %
0.65 %
0.78 %
0.91 %
1.04 %
1.17 %
1.3 %
1.43 %
1.56 %
1.69 %
1.82 %
1.95 %
2.08 %
2.21 %
2.34 %
2.47 %
2.6 %
2.73 %
2.86 %
2.99 %
3.12 %
3.25 %
3.38 %
3.51 %
3.63 %
3.76 %
3.89 %
4.02 %
4.15 %
4.28 %
4.41 %
4.54 %
4.67 %
4.8 %
4.93 %
5.06 %
5.19 %
5.32 %
5.45 %
5.58 %
5.71 %
5.84 %
5.97 %
6.1 %
6.23 %
6.36 %
6.49 %
6.62 %
6.75 %
6.88 %
7.01 %
7.14 %
7.27 %
7.4 %
7.53 %
7.66 %
7.79 %
7.92 %
8.05 %
8.18 %
8.31 %
8.44 %
8.57 %
8.7 %
8.83 %
8.96 %
9.09 %
9.22 %
9.35 %
9.48 %
9.61 %
9.74 %
9.87 %
10.0 %
10.13 %
10.26 %
10.39 %
10.52 %
10.65 %
10.78 %
10.9 %
11.03 %
11.16 %
11.29 %
11.42 %
11.55 %
11.68 %
11.81 %
11.94 %
12.07 %
12.2 %
12.33 %
12.46 %
12.59 %
12.72 %
12.85 %
12.98 %
13.11 %
13.24 %
13.37 %
13.5 %
13.63 %
13.76 %
13.89 %
14.02 %
14.15 %
14.28 %
14.41 %
14.54 %
14.67 %
14.8 %
14.93 %
15.06 %
15.19 %
15.32 %
15.45 %
15.58 %
15.71 %
15.84 %
15.97 %
16.1 %
16.23 %
16.36 %
16.49 %
16.62 %
16.75 %
16.88 %
17.01 %
17.14 %
17.27 %
17.4 %
17.53 %
17.66 %
1

In [83]:
#更改時區時間至台灣時間

def getTWtime(num):
   
    time_append['time'][num]=frame['latest'][num].astimezone(tz)

    return 0

def get_areatimebythread(totaldata):
    totalnum=totaldata  #總執行次數
    Q=int(totalnum/10) #取商數
    R=totalnum%10     #取餘數
    
    for i in range(Q):
        threads = []
        for j in range(10):#開多少執行緒
            threads.append(threading.Thread(
                target=getTWtime,#要執行函數
                args=(i*10+j)))#要執行函數的參數
            threads[j].start()
        for j in threads:
            j.join()
        print(round(float((i+1)*100/totalnum*10),2),'%')#顯示進度
    
    threads = []
    for i in range(R):
        threads.append(threading.Thread(
            target=getTWtime,
            args=(frame,Q*10+i)))
        threads[i].start()
    for j in threads:
        j.join()
    print("100%")#顯示進度


tz = pytz.timezone('Asia/Taipei')
time_append = pd.DataFrame(columns=['time'])
time_append['time'] = frame['latest']   

t1=time.time()
get_areatimebythread(frame.shape[0])
t2=time.time()
print('gevent-time:%s' % str(t2-t1))
frame['latest']=time_append['time']


Exception in thread Thread-47185:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-47186:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
Exception in thread Thread-47187:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception in thread   File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
Thread-47188Exception in thread Thread-47189:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
:
    Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception 

Exception in thread Exception in thread Thread-47208:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Thread-47207:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    Exception in thread Thread-47209:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
Exception in thread     Thread-47210    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
:
Exception in thread Thread-47211:
Traceback (most recent call last):
  File "D:\Use

0.13 %
0.26 %


    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
Exception in thread Thread-47215:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * m

0.39 %
0.52 %


...

99.44 %
99.57 %


    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
Exception in thread Thread-54863:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-54864:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
Exception in thread Thread-54

99.7 %
99.83 %


    self.run()
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
TypeError: __main__.getTWtime() argument after * must be an iterable, not int
Exception in thread Thread-54881:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-54882:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
Exception in thread Thread-54883:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner
    Exception in thread self.run()Thread-54884:
Traceback (most recent call last):
  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 954, in _bootstrap_inner

  File "D:\Users\jack5\Anaconda3\envs\geo\lib\threading.py", line 892, in run
    self.run()
  File "D:\Users\jack5\Anaconda3\en

99.96 %
100%
gevent-time:87.5664873123169


In [84]:
frame.drop(['humidity','temperature','ranking','site','ver_app','malfunction_1','malfunction_2','malfunction_3','pollution','source','survey'],axis=1,inplace=True)
#轉換時間格式
frame['latest'] = frame['latest'].apply(lambda x:time.mktime(x.timetuple()))#x.timetuple()將時間轉換為時間元組，提前匯入time模組
frame['latest'] = frame['latest'].apply(lambda x:time.strftime('%Y/%m/%d %H:%M:%S',time.localtime(x)))#程式碼含義為：先讀取時間戳資料，將其轉換為時間元組，在通過格式化時間轉換為可讀的時間格式
frame

,pm25,site_id,latest,longitude,latitude,areaname,Id
0,8.0,EPAIoT_9999949564,2021/02/20 17:43:00,121.370470,25.082390,新北市林口區,1.0
1,16.0,EPAIoT_9999844140,2021/02/20 17:43:00,121.393750,24.941340,新北市三峽區,2.0
2,30.0,EPAIoT_9999729712,2021/02/20 17:43:00,121.438340,24.974660,新北市土城區,3.0
3,3.0,EPAIoT_9999615935,2021/02/20 17:43:00,121.497160,25.002440,新北市中和區,4.0
4,22.0,EPAIoT_9999597074,2021/02/20 17:43:00,121.392873,24.940336,新北市三峽區,5.0
...,...,...,...,...,...,...,...
7698,25.0,EPAIoT_10005897410,2021/02/20 17:42:00,120.434670,23.519510,嘉義縣民雄鄉,8543.0
7699,25.0,EPAIoT_10000399488,2021/02/20 17:42:00,120.429510,23.522140,嘉義縣民雄鄉,8544.0
7700,29.0,EPAIoT_10000275182,2021/02/20 17:42:00,120.430480,23.522070,嘉義縣民雄鄉,8545.0
7701,35.0,EPAIoT_10000167840,2021/02/20 17:43:00,121.457070,24.996440,新北市板橋區,8546.0


In [86]:
epa = frame.loc[frame["areaname"].str.contains('台中|臺中')]
epa=epa.drop(columns=['latest']).reset_index(drop=True)
epa.index= range(1,len(epa) + 1)
epa['Id']=epa.index
epa.index= range(0,len(epa))

epa=epa[['site_id','Id','pm25','latitude','longitude']]


epa.rename(columns={ epa.columns[0]: "SiteName"}, inplace=True)
epa.rename(columns={ epa.columns[2]: "PM2.5"}, inplace=True)
epa.rename(columns={ epa.columns[3]: "Latitude"}, inplace=True)
epa.rename(columns={ epa.columns[4]: "Longitude"}, inplace=True)
epaidw=epa
epaidw=epaidw.drop(columns=['SiteName']).reset_index(drop=True)
epaidw

,Id,PM2.5,Latitude,Longitude
0,1,32.000000,24.159462,120.655411
1,2,44.880001,24.177469,120.576571
2,3,26.000000,24.159441,120.655304
3,4,31.000000,24.158655,120.656578
4,5,34.000000,24.158928,120.656548
...,...,...,...,...
1263,1264,30.670000,24.173178,120.675928
1264,1265,30.090000,24.151993,120.649922
1265,1266,33.299999,24.168196,120.695632
1266,1267,34.470001,24.161635,120.679379


In [87]:
frame

,pm25,site_id,latest,longitude,latitude,areaname,Id
0,8.0,EPAIoT_9999949564,2021/02/20 17:43:00,121.370470,25.082390,新北市林口區,1.0
1,16.0,EPAIoT_9999844140,2021/02/20 17:43:00,121.393750,24.941340,新北市三峽區,2.0
2,30.0,EPAIoT_9999729712,2021/02/20 17:43:00,121.438340,24.974660,新北市土城區,3.0
3,3.0,EPAIoT_9999615935,2021/02/20 17:43:00,121.497160,25.002440,新北市中和區,4.0
4,22.0,EPAIoT_9999597074,2021/02/20 17:43:00,121.392873,24.940336,新北市三峽區,5.0
...,...,...,...,...,...,...,...
7698,25.0,EPAIoT_10005897410,2021/02/20 17:42:00,120.434670,23.519510,嘉義縣民雄鄉,8543.0
7699,25.0,EPAIoT_10000399488,2021/02/20 17:42:00,120.429510,23.522140,嘉義縣民雄鄉,8544.0
7700,29.0,EPAIoT_10000275182,2021/02/20 17:42:00,120.430480,23.522070,嘉義縣民雄鄉,8545.0
7701,35.0,EPAIoT_10000167840,2021/02/20 17:43:00,121.457070,24.996440,新北市板橋區,8546.0


In [88]:
time_append

,time
0,2021-02-20 17:43:00+00:00
1,2021-02-20 17:43:00+00:00
2,2021-02-20 17:43:00+00:00
3,2021-02-20 17:43:00+00:00
4,2021-02-20 17:43:00+00:00
...,...
7698,2021-02-20 17:42:00+00:00
7699,2021-02-20 17:42:00+00:00
7700,2021-02-20 17:42:00+00:00
7701,2021-02-20 17:43:00+00:00


In [89]:
frame

,pm25,site_id,latest,longitude,latitude,areaname,Id
0,8.0,EPAIoT_9999949564,2021/02/20 17:43:00,121.370470,25.082390,新北市林口區,1.0
1,16.0,EPAIoT_9999844140,2021/02/20 17:43:00,121.393750,24.941340,新北市三峽區,2.0
2,30.0,EPAIoT_9999729712,2021/02/20 17:43:00,121.438340,24.974660,新北市土城區,3.0
3,3.0,EPAIoT_9999615935,2021/02/20 17:43:00,121.497160,25.002440,新北市中和區,4.0
4,22.0,EPAIoT_9999597074,2021/02/20 17:43:00,121.392873,24.940336,新北市三峽區,5.0
...,...,...,...,...,...,...,...
7698,25.0,EPAIoT_10005897410,2021/02/20 17:42:00,120.434670,23.519510,嘉義縣民雄鄉,8543.0
7699,25.0,EPAIoT_10000399488,2021/02/20 17:42:00,120.429510,23.522140,嘉義縣民雄鄉,8544.0
7700,29.0,EPAIoT_10000275182,2021/02/20 17:42:00,120.430480,23.522070,嘉義縣民雄鄉,8545.0
7701,35.0,EPAIoT_10000167840,2021/02/20 17:43:00,121.457070,24.996440,新北市板橋區,8546.0


In [91]:
frame['pm25'].replace({"NA":np.nan}, inplace=True)
frame['pm25'].astype('float64')
frame['pm25'].replace({"-99":np.nan,"-99.0":np.nan}, inplace=True)
frame[frame['pm25']<0]=np.nan
frame.dropna(inplace=True)

frame

,pm25,site_id,latest,longitude,latitude,areaname,Id
0,8.0,EPAIoT_9999949564,2021/02/20 17:43:00,121.370470,25.082390,新北市林口區,1.0
1,16.0,EPAIoT_9999844140,2021/02/20 17:43:00,121.393750,24.941340,新北市三峽區,2.0
2,30.0,EPAIoT_9999729712,2021/02/20 17:43:00,121.438340,24.974660,新北市土城區,3.0
3,3.0,EPAIoT_9999615935,2021/02/20 17:43:00,121.497160,25.002440,新北市中和區,4.0
4,22.0,EPAIoT_9999597074,2021/02/20 17:43:00,121.392873,24.940336,新北市三峽區,5.0
...,...,...,...,...,...,...,...
7698,25.0,EPAIoT_10005897410,2021/02/20 17:42:00,120.434670,23.519510,嘉義縣民雄鄉,8543.0
7699,25.0,EPAIoT_10000399488,2021/02/20 17:42:00,120.429510,23.522140,嘉義縣民雄鄉,8544.0
7700,29.0,EPAIoT_10000275182,2021/02/20 17:42:00,120.430480,23.522070,嘉義縣民雄鄉,8545.0
7701,35.0,EPAIoT_10000167840,2021/02/20 17:43:00,121.457070,24.996440,新北市板橋區,8546.0


In [92]:
#now_time=(datetime.datetime.now()+datetime.timedelta(hours=0)).strftime("%Y-%m-%d %H")
#now_time

In [93]:
#now_time=(datetime.datetime.now()+datetime.timedelta(hours=0)).strftime("%Y-%m-%d %H")
#frame=frame.loc[frame["datetime"].str.contains(now_time)]
frame.to_csv('E:\\GitHub\\Independent-study\\web\\123321\\taiwan_all_micro.csv')
frame

,pm25,site_id,latest,longitude,latitude,areaname,Id
0,8.0,EPAIoT_9999949564,2021/02/20 17:43:00,121.370470,25.082390,新北市林口區,1.0
1,16.0,EPAIoT_9999844140,2021/02/20 17:43:00,121.393750,24.941340,新北市三峽區,2.0
2,30.0,EPAIoT_9999729712,2021/02/20 17:43:00,121.438340,24.974660,新北市土城區,3.0
3,3.0,EPAIoT_9999615935,2021/02/20 17:43:00,121.497160,25.002440,新北市中和區,4.0
4,22.0,EPAIoT_9999597074,2021/02/20 17:43:00,121.392873,24.940336,新北市三峽區,5.0
...,...,...,...,...,...,...,...
7698,25.0,EPAIoT_10005897410,2021/02/20 17:42:00,120.434670,23.519510,嘉義縣民雄鄉,8543.0
7699,25.0,EPAIoT_10000399488,2021/02/20 17:42:00,120.429510,23.522140,嘉義縣民雄鄉,8544.0
7700,29.0,EPAIoT_10000275182,2021/02/20 17:42:00,120.430480,23.522070,嘉義縣民雄鄉,8545.0
7701,35.0,EPAIoT_10000167840,2021/02/20 17:43:00,121.457070,24.996440,新北市板橋區,8546.0


In [94]:

epa

,SiteName,Id,PM2.5,Latitude,Longitude
0,EPAIoT_8009702884,1,32.000000,24.159462,120.655411
1,EPAIoT_7782112149,2,44.880001,24.177469,120.576571
2,EPAIoT_7779666383,3,26.000000,24.159441,120.655304
3,EPAIoT_7663138797,4,31.000000,24.158655,120.656578
4,EPAIoT_7662938612,5,34.000000,24.158928,120.656548
...,...,...,...,...,...
1263,EPAIoT_10232916032,1264,30.670000,24.173178,120.675928
1264,EPAIoT_10232839597,1265,30.090000,24.151993,120.649922
1265,EPAIoT_10232760701,1266,33.299999,24.168196,120.695632
1266,EPAIoT_10232655835,1267,34.470001,24.161635,120.679379


In [95]:
#epaidw.columns=['site_id','latitude','longitude','pm25','Id']
epaidw=epaidw[['Id','Latitude','Longitude','PM2.5']]

epaidw

,Id,Latitude,Longitude,PM2.5
0,1,24.159462,120.655411,32.000000
1,2,24.177469,120.576571,44.880001
2,3,24.159441,120.655304,26.000000
3,4,24.158655,120.656578,31.000000
4,5,24.158928,120.656548,34.000000
...,...,...,...,...
1263,1264,24.173178,120.675928,30.670000
1264,1265,24.151993,120.649922,30.090000
1265,1266,24.168196,120.695632,33.299999
1266,1267,24.161635,120.679379,34.470001


In [96]:
# print(times[21:36]) 時間
df1 = pd.read_html(data1.text)[1]
cols1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df1 = df1.replace('─','0')
df1 = df1.drop(df1.columns[cols1],axis=1)
df1.rename(columns={ df1.columns[0]: "SiteName"}, inplace=True)
df1.rename(columns={ df1.columns[1]: "PM2.5"}, inplace=True)
df1['Latitude']=[24.1622,24.151958,24.099611,24.225628,
                 24.256586,24.139008,24.350426,24.139564,
                 24.05735,24.094264,24.307036,24.250388,
                 24.150919,24.182055,24.269233,24.20103]

df1['Longitude']=[120.616917,120.641092,120.677689,120.568794,
                  120.741711,120.597876,120.615358,120.715064,
                  120.697299,120.646629,120.714881,120.538839,
                  120.540877,120.60707,120.576421,120.498566]


In [97]:
df1.rename(columns={"測站名稱":"SiteName"}, inplace=True)
df1.rename(columns={ "細懸浮微粒(PM2.5)":"PM2.5"}, inplace=True)
df1

,SiteName,PM2.5,Latitude,Longitude
,SiteName,移動平均濃度(μg/m3),,
0,西屯測站(環保署),36,24.162200,120.616917
1,忠明測站(環保署),37,24.151958,120.641092
2,大里測站(環保署),36,24.099611,120.677689
3,沙鹿測站(環保署),32,24.225628,120.568794
4,豐原測站(環保署),30,24.256586,120.741711
5,文山測站,34,24.139008,120.597876
6,大甲測站,33,24.350426,120.615358
7,太平測站,31,24.139564,120.715064
8,霧峰測站,30,24.057350,120.697299


In [98]:
df1[['Latitude', 'Longitude', 'PM2.5']] = df1[['Latitude', 'Longitude','PM2.5']].astype(float)
#df1  顯示16筆測站
df1.to_csv("E:\\GitHub\\Independent-study\\web\\123321\\df1_for_now20.csv")
df1 = pd.read_csv("E:\\GitHub\\Independent-study\\web\\123321\\df1_for_now20.csv")
df1.drop([0],inplace=True) #把df1不要的第一row砍掉
#pm25_predict_answer=df1
df11=df1['SiteName']       #先測站把名稱存進df11
df1.drop(["SiteName"],axis=1,inplace=True)#測站名稱column砍掉


In [99]:
df1

,Unnamed: 0,PM2.5,Latitude,Longitude
1,0.0,36.0,24.162200,120.616917
2,1.0,37.0,24.151958,120.641092
3,2.0,36.0,24.099611,120.677689
4,3.0,32.0,24.225628,120.568794
5,4.0,30.0,24.256586,120.741711
6,5.0,34.0,24.139008,120.597876
7,6.0,33.0,24.350426,120.615358
8,7.0,31.0,24.139564,120.715064
9,8.0,30.0,24.057350,120.697299
10,9.0,35.0,24.094264,120.646629


In [101]:
#pm25_predict_answer.rename({'Unnamed: 0':'time'}, inplace=True)
#pm25_predict_answer['time']=now_time

df11=df11.append(epa['SiteName'])#在把微型感測器測站名稱也併進df11
df1.rename(columns={ 'Unnamed: 0':'Id'}, inplace=True)#更改unname--->Id
#df1=df1[['Id','site_id','PM2.5','Latitude','Longitude']] df1目前型態
df1=df1.append(epaidw) #把微型感測器資料也併進來
df1=df1.astype('float64')#轉成float64
df11.index= range(0,len(df11))#index重排
df1.index=range(0,len(df1))

In [102]:
#taichung = gp.read_file("/home/hpc/taichungcity.geojson")           #台中邊界
taichungmap_1x1 = gp.read_file("E:\\GitHub\\Independent-study\\web\\123321\\final.geojson")         #台中1*1網格
taichung_district = gp.read_file("E:\\GitHub\\Independent-study\\web\\123321\\taichung_district.geojson")
#list1= [   1,    4,   14,   26,   44,   63,   82,  102,  122,  144,
#         168,  193,  221,  257,  304,  353,  403,  455,  510,  568,
#         627,  687,  750,  819,  892,  968, 1053, 1141, 1232, 1325,
#        1418, 1510, 1601, 1692, 1781, 1864, 1944, 2019, 2087, 2145,
#        2197, 2246, 2289, 2330, 2359, 2384, 2403, 2419, 2433, 2445 ]
#list2= [   3,   13,   25,   43,   62,   81,  101,  121,  143,  167,
#         192,  220,  256,  303,  352,  402,  454,  509,  567,  626,
#         686,  749,  818,  891,  967, 1052, 1140, 1231, 1324, 1417,
#        1509, 1600, 1691, 1780, 1863, 1943, 2018, 2086, 2144, 2196,
#        2245, 2288, 2329, 2358, 2383, 2402, 2418, 2432, 2444, 2449 ]
lon_max=taichungmap_1x1.bounds.maxx
lon_min=taichungmap_1x1.bounds.minx
lat_max=taichungmap_1x1.bounds.maxy
lat_min=taichungmap_1x1.bounds.miny

In [103]:
print(df1.shape)
df1.head()

(1284, 4)


,Id,PM2.5,Latitude,Longitude
0,0.0,36.0,24.162200,120.616917
1,1.0,37.0,24.151958,120.641092
2,2.0,36.0,24.099611,120.677689
3,3.0,32.0,24.225628,120.568794
4,4.0,30.0,24.256586,120.741711


In [104]:
#lat_max,lat_min,lon_max,lon_min四份合併做成DataFrame
df_grid = pd.DataFrame([lat_max,lat_min,lon_max,lon_min]).T
df_grid['grid_Longitude'] = (df_grid['maxx']+df_grid['minx'])/2
df_grid['grid_Latitude']  = (df_grid['maxy']+df_grid['miny'])/2
df_grid['grid_ID']   = df_grid.index+1
df_grid

,maxy,miny,maxx,minx,grid_Longitude,grid_Latitude,grid_ID
0,24.004430,23.995383,120.744002,120.734155,120.739079,23.999907,1
1,24.004446,23.995400,120.753831,120.743984,120.748907,23.999923,2
2,24.004461,23.995416,120.763659,120.753813,120.758736,23.999939,3
3,24.013349,24.004299,120.685010,120.675158,120.680084,24.008824,4
4,24.013369,24.004320,120.694839,120.684988,120.689914,24.008844,5
...,...,...,...,...,...,...,...
2444,24.446614,24.437560,120.624769,120.614879,120.619824,24.442087,2445
2445,24.446638,24.437585,120.634631,120.624742,120.629687,24.442112,2446
2446,24.446863,24.437817,121.265845,121.255964,121.260905,24.442340,2447
2447,24.446772,24.437722,121.315161,121.305276,121.310218,24.442247,2448


In [105]:
#定義IDW

def idw(data_obs,data_grid,radius,interval):
    print('半徑 %.2Fkm 範圍內無觀測站則無法觀測'%(radius+interval))#*2
    df_goal = pd.DataFrame([])
    for grid_i in range(len(data_grid)):
        data_obs['grid_ID'] = data_grid.loc[grid_i,'grid_ID']
        df_work = pd.merge(data_grid,data_obs,how='inner',on='grid_ID')
    
        size= 0.0092
    
        df_work['Lat_dis_km']   = abs((df_work['Latitude']  - df_work['grid_Latitude']) /size)
        df_work['Lon_dis_km']   = abs((df_work['Longitude'] - df_work['grid_Longitude'])/size)
        df_work['distance']     = np.sqrt(df_work['Lat_dis_km']**2+df_work['Lon_dis_km']**2)
        df_work['distance_rec'] = 1/df_work['distance']

        df_work = df_work.sort_values(by=['distance'])
        df_work = df_work.reset_index()

 
        #做出圓心
        center_list = df_work.loc[df_work['distance']<=radius]
        df_center = pd.DataFrame([])
        if len(center_list)==0:
            df_center = pd.DataFrame({'grid_ID':[grid_i+1],'center_PM2.5':[None]})
        else:
            center_temp = center_list.groupby('grid_ID')['distance_rec'].sum()
            center_list = pd.merge(center_list,center_temp,how='inner',on='grid_ID')
            center_list['center_PM2.5']=(center_list['distance_rec_x']/center_list['distance_rec_y'])*center_list['PM2.5']
            df_center = pd.DataFrame(center_list.groupby(['grid_ID'])['center_PM2.5'].sum()).reset_index()  
    
        #做出第1層圓環
        L1_list = df_work.loc[(df_work['distance']>radius)&(df_work['distance']<=(radius+interval))]
        df_L1 = pd.DataFrame([])
        if len(L1_list)==0:
            df_L1 = pd.DataFrame({'grid_ID':[grid_i+1],'L1_PM2.5':[None]})
        else:
            L1_temp = L1_list.groupby('grid_ID')['distance_rec'].sum()
            L1_list = pd.merge(L1_list,L1_temp,how='inner',on='grid_ID')
            L1_list['L1_PM2.5']=(L1_list['distance_rec_x']/L1_list['distance_rec_y'])*L1_list['PM2.5']
            df_L1 = pd.DataFrame(L1_list.groupby(['grid_ID'])['L1_PM2.5'].sum()).reset_index()
 
        #做出第2層圓環
        L2_list = df_work.loc[(df_work['distance']>(radius+interval))&(df_work['distance']<=(radius+interval*2))]
        df_L2 = pd.DataFrame([])
        if len(L2_list)==0:
             df_L2 = pd.DataFrame({'grid_ID':[grid_i+1],'L2_PM2.5':[None]})
        else:
            L2_temp = L2_list.groupby('grid_ID')['distance_rec'].sum()
            L2_list = pd.merge(L2_list,L2_temp,how='inner',on='grid_ID')
            L2_list['L2_PM2.5']=(L2_list['distance_rec_x']/L2_list['distance_rec_y'])*L2_list['PM2.5']
            df_L2 = pd.DataFrame(L2_list.groupby(['grid_ID'])['L2_PM2.5'].sum()).reset_index()

    
        #合併數據(join)
        df_all = pd.merge(pd.merge(df_center,df_L1,on='grid_ID'),df_L2,on='grid_ID')
        df_all['radius'] = radius
        df_all['interval'] = interval

        #總合併(union)
        df_goal = pd.concat([df_goal,df_all],sort=True)
    return df_goal

In [106]:
#執行idw()並deepcopy切割
df_goal = copy.deepcopy(idw(df1,df_grid,5,5))

半徑 15.00km 範圍內無觀測站則無法觀測


In [107]:
df_goal.reset_index(inplace=True)

In [108]:
#製作狀況D的圓心推估值(center_adj)
df_goal['dis_weight'] = ((df_goal['interval']*1.5+df_goal['radius'])/(df_goal['interval']*0.5+df_goal['radius']))-1
df_goal['adj'] = (df_goal['L2_PM2.5']-df_goal['L1_PM2.5'])
df_goal['center_adj']   =  df_goal['L1_PM2.5']-df_goal['adj']*df_goal['dis_weight']

In [109]:
df_goal

,index,L1_PM2.5,L2_PM2.5,center_PM2.5,grid_ID,interval,radius,dis_weight,adj,center_adj
0,0,31.470413,37.067818,41.209999,1,10,5,1.0,5.597405,25.873008
1,0,31.073553,36.710346,NaN,2,10,5,1.0,5.636793,25.436760
2,0,30.900712,36.413070,NaN,3,10,5,1.0,5.512358,25.388354
3,0,31.632242,38.753989,39.863243,4,10,5,1.0,7.121747,24.510495
4,0,31.478060,38.872023,39.377416,5,10,5,1.0,7.393962,24.084098
...,...,...,...,...,...,...,...,...,...,...
2444,0,31.003376,32.823976,28.170842,2445,10,5,1.0,1.820599,29.182777
2445,0,30.169481,32.965034,30.029441,2446,10,5,1.0,2.795553,27.373928
2446,0,6.000000,4.920000,NaN,2447,10,5,1.0,-1.080000,7.080000
2447,0,6.000000,4.920000,NaN,2448,10,5,1.0,-1.080000,7.080000


In [110]:
'''
依照狀況A~D，給上不同的最終推估值(est_PM2.5)
共有以下A~D四種狀況：
A.半徑5km內至少有1個觀測站，則估計值直接使用center_PM2.5。
B.半徑5km內沒有任何觀測站、第一層圓環至少有1觀測站、第二層圓環沒有任何觀測站，則估計值直接使用L1_PM2.5。
C.半徑5km內沒有任何觀測站、第一層圓環沒有任何觀測站、第二層圓環至少有1觀測站，則估計值無法估計。
D.半徑5km內沒有任何觀測站、第一層圓環至少有1觀測站、第二層圓環至少有1觀測站，則使用L1_PM2.5和L2_PM2.5的遞減遞增估計。
E.第二圓環以內完全沒有任何觀測站，即半徑25km以內完全沒有觀測站，則不應該估計數值：無法估計。
'''
condition_A =  ~df_goal['center_PM2.5'].isnull()
condition_B = ( df_goal['center_PM2.5'].isnull())&(~df_goal['L1_PM2.5'].isnull())&( df_goal['L2_PM2.5'].isnull())
condition_C = ( df_goal['center_PM2.5'].isnull())&( df_goal['L1_PM2.5'].isnull())&(~df_goal['L2_PM2.5'].isnull())
condition_D = ( df_goal['center_PM2.5'].isnull())&(~df_goal['L1_PM2.5'].isnull())&(~df_goal['L2_PM2.5'].isnull())
        
df_goal['est_PM2.5'] = df_goal['center_PM2.5']
df_goal['est_PM2.5'].loc[condition_A]=df_goal['center_PM2.5'].loc[condition_A]
df_goal['est_PM2.5'].loc[condition_B]=df_goal['L1_PM2.5'].loc[condition_B]
df_goal['est_PM2.5'].loc[condition_C]=-1
#df_goal['est_PM2.5'].loc[condition_C]=df_goal['L2_PM2.5'].loc[condition_C]
df_goal['est_PM2.5'].loc[condition_D]=df_goal['center_adj'].loc[condition_D]


D:\Users\jack5\Anaconda3\envs\geo\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [111]:
#產出df3以供後續應用
df3 = pd.merge(df_goal,df_grid,how='inner',on='grid_ID')
df3 = df3[['grid_Latitude','grid_Longitude','est_PM2.5','grid_ID']]
df3.columns = ['Latitude','Longitude','PM2.5','Id']
df3.loc[df3['PM2.5'].isnull(),'PM2.5']=-1
df3[df3['PM2.5']<0]=-1

In [112]:
df3.to_csv("E:\\GitHub\\Independent-study\\web\\123321\\all_point_data_epa.csv")
df3.to_csv("E:\\GitHub\\Independent-study\\web\\123321\\testidw2_all.csv")
all_point_data_epa = pd.read_csv("E:\\GitHub\\Independent-study\\web\\123321\\all_point_data_epa.csv")
lon=list(all_point_data_epa['Longitude'])
lat=list(all_point_data_epa['Latitude'])
#all_point_data_epa['Id']=0
Id=taichungmap_1x1['Id']
ans_Id=all_point_data_epa['Id']

In [113]:
def generalID(lon,lat,column_num,row_num):
    # 若在范围外的点，返回-1
    if lon <= LON1 or lon >= LON2 or lat <= LAT1 or lat >= LAT2:
        return -1
    # 把经度范围根据列数等分切割
    column = (LON2 - LON1)/column_num
    # 把纬度范围根据行数数等分切割
    row = (LAT2 - LAT1)/row_num
    # 二维矩阵坐标索引转换为一维ID，即： （列坐标区域（向下取整）+ 1） + （行坐标区域 * 列数）
    return int((lon-LON1)/column)+ 1 + int((lat-LAT1)/row) * column_num


In [114]:
# fit id
#for i in range(2449):
#    ans_Id[i] = i+1
# for d in (range(100)):      #測試資料筆數
#      for m in (range(50)):  #找是在哪列
#             if lon_min[list1[m]-1]<lon[d] and lon[d]<lon_max[list2[m]-1] and lat_min[list1[m]-1]<lat[d] and lat[d]<lat_max[list2[m]-1]:
#                 for n in range(list1[m],list2[m]+1): #找是在該列的哪個
#                     if lon_min[n-1]<lon[d] and lon[d]<lon_max[n-1] and lat_min[n-1]<lat[d] and lat[d]<lat_max[n-1]:
#                         ans_Id[d]=Id[n]-1

#taichung_district

In [115]:
taichungmap_1x1 = taichungmap_1x1.merge(all_point_data_epa, on='Id')
taichungmap_1x1['PM2.5']=taichungmap_1x1['PM2.5'].round()



In [116]:
df1['SiteName']=df11
df1=df1[['Id','SiteName','PM2.5','Latitude','Longitude']]


In [121]:
# =============================================================================================================
# folium
#(215, 207, 207, 0.00)
variable = 'PM2.5'
colorList = [(215, 207, 207, 1), '#98fb98', '#51ff51', '#00ff00', '#1ce11c',
             '#32cd32', '#ffff00', '#ffee00', '#ffd13f', '#ffc700', '#ffbf4a',
             '#ffa500', '#ff6347', '#ff5047', '#ff4c2c', '#ff0000', '#d32c4a',
             '#ba55d3']
map_color = cm.StepColormap(colorList,index=[-1,0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
    vmin=-1,vmax=100,caption = 'PM2.5')

fmap = folium.Map(location=[24.2, 120.9], zoom_start=10.5)
# fmap.choropleth(
#                geo_data=taichungmap_1x1,
#                name='pm2.5',
#                columns=['Id', 'PM2.5'],
#                key_on='feature.properties.Id',
#                data=all_point_data_epa,
#               #threshold_scale=[],
#                fill_color='BuGn',
#                legend_name='pm2.5',
#                line_opacity=0.5,
#                fill_opacity=0.8
#                )
folium.GeoJson(taichungmap_1x1,
               name='PM2.5',
               style_function=lambda x: {
                   'fillColor': map_color(x['properties'][variable]),
                   'color': 'black',
                   'weight': 0,
                   'fillOpacity': 0.7
               },
               highlight_function=lambda x: {
                   'weight': 3,
                   'color': 'black'
               },
               tooltip=folium.GeoJsonTooltip(fields=['Id', 'PM2.5'],
                                             aliases=['Id', 'PM2.5'],
                                             labels=True,
                                             sticky=True)).add_to(fmap)

#微型感測器logo
epa_micro_url= 'https://ci.taiwan.gov.tw/dsp/img/map_icon/air_quality.png'
# 環保署 logo
epa_icon_url = 'https://www.epa.gov.tw/public/MMO/epa/Epa_Logo_01_LOGO.gif'

station = folium.FeatureGroup(name="環保署", show=True)
for i in (range(15)):
    station.add_child(
        folium.Marker(
            location=[df1['Latitude'][i], df1['Longitude'][i]],
            popup=("<b>NAME:</b> {NAME}<br>"
                   " <p><b>PM2.5:</b> {PM25}<br>"
                   " <p><b>TIME:</b> {TIME}<br>").format(
                       NAME=str(df1['SiteName'][i]),
                       PM25=str(df1['PM2.5'][i]),
                       TIME=str(times[39:55])),
            icon=folium.CustomIcon(epa_icon_url,
                                   icon_size=(23,
                                              23))  # Creating a custom Icon
        ))
epa_micro = folium.FeatureGroup(name="微型感測器", show=True)
for k in (range(epa.shape[0]-1)):
    epa_micro.add_child(
        folium.Marker(
            location=[df1['Latitude'][k + 16], df1['Longitude'][k + 16]],
            popup=("<b>NAME:</b> {NAME}<br>"
                   " <p><b>PM2.5:</b> {PM25}<br>"
                   " <p><b>TIME:</b> {TIME}<br>").format(
                       NAME=str(df1['SiteName'][k + 16]),
                       PM25=str(df1['PM2.5'][k + 16]),
                       TIME=str(times_micro[:][k])),
            icon=folium.CustomIcon(epa_micro_url,
                                   icon_size=(15,15))  # Creating a custom Icon
        ))


fmap.add_child(epa_micro)
fmap.add_child(station)

fmap.add_child(map_color)
folium.LayerControl().add_to(fmap)
# lat/lon to map
# folium.LatLngPopup().add_to(fmap)
fmap.save('E:\\GitHub\\Independent-study\\web\\123321\\pm25_for_now20.html')  # 存成 final.html

In [122]:
 times_micro

0       2021/02/20 17:43:00
1       2021/02/20 17:43:00
2       2021/02/20 17:43:00
3       2021/02/20 17:43:00
4       2021/02/20 17:43:00
               ...         
7698    2021/02/20 17:42:00
7699    2021/02/20 17:42:00
7700    2021/02/20 17:42:00
7701    2021/02/20 17:43:00
7702    2021/02/20 17:42:00
Name: latest, Length: 7703, dtype: object